<a href="https://colab.research.google.com/github/jayaraman1112g/ai_snippets/blob/main/llamaindex_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU llama-index
!pip install -qU transformers
!pip install -qU sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.7 MB/s eta 0:00:00


In [2]:
%pip install -qU llama-index-embeddings-huggingface
%pip install -qU llama-index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00


# Read the document

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings


documents = SimpleDirectoryReader(
    input_files=["sample_data/DISEASE.pdf"]
).load_data()

# Setup the LLM

In [6]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # loading model in 8bit for reducing memory
    model_kwargs={"torch_dtype": torch.float16 }
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
embed_model = HuggingFaceEmbedding (model_name='BAAI/bge-small-en-v1.5',token=hf_token )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from llama_index.core import ServiceContext
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)
index = VectorStoreIndex.from_documents (documents, service_context = service_context)

<ipython-input-8-cc293b757791>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [9]:
from llama_index.core.response.notebook_utils import display_source_node
retriever = index.as_retriever ()
nodes = retriever.retrieve ("What is diphtheria?")
for node in nodes:
    display_source_node(node)

**Node ID:** 91343053-c58f-48c9-af22-088ac7945bc8<br>**Similarity:** 0.6850523838819751<br>**Text:** Further doses of diphtheria-tetanus vaccine are recommended every ten years.  Protection can be v...<br>

**Node ID:** 10829107-6efd-48fd-811f-1394b74de3b5<br>**Similarity:** 0.6436952170970844<br>**Text:** 10 
10 A form of diphtheria which involves the skin, eyes or genitals also exists.  Complications...<br>

In [10]:
qe = index.as_query_engine ()
nodes = qe.query ("What is diptheria")
print (nodes)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Diptheria is a bacterial infection caused by Corynebacterium diphtheriae. It can cause a severe respiratory infection, including difficulty breathing, and can lead to serious complications such as heart problems and paralysis. Further doses of diphtheria-tetanus vaccine are recommended every ten years to protect against the disease. Protection can be verified by measuring the antitoxin level in the blood.
